In [ ]:
import os
import cv2
import numpy as np
import glob

In [ ]:
#Paths for datasets
base_dataset_path = '/kaggle/input/deforestation-in-ukraine/'
development_dataset_path = '/kaggle/working/processed_dataset/'

**This function iterates through the dataset, preprocesses each .jp2 image (resize and converts images from .jp2 to .jpg format) and saves the processed images in a new directory.**

In [ ]:
def preprocess_and_save_images(base_dataset_path, dev_dataset_path, target_size=(256, 256)):
    
    if not os.path.exists(dev_dataset_path):
        os.makedirs(dev_dataset_path)

    #Itirate through all subfolders and convert all .jp2 images
    for folder in os.listdir(base_dataset_path):
        img_data_path = os.path.join(base_dataset_path, folder, "*.SAFE/GRANULE/*/IMG_DATA/*_TCI.jp2")
        tci_image_files = glob.glob(img_data_path)
        
        for image_file in tci_image_files:
            subdirectory_name = image_file.split('/')[-3]
            dest_subdirectory_path = os.path.join(dev_dataset_path, subdirectory_name)
            
            if not os.path.exists(dest_subdirectory_path):
                os.makedirs(dest_subdirectory_path)

            # Load and preprocess image
            preprocessed_image = load_and_preprocess_image(image_file, target_size)
            dest_image_path = os.path.join(dest_subdirectory_path, os.path.basename(image_file).replace('.jp2', '.jpg'))
            
            cv2.imwrite(dest_image_path, preprocessed_image)

def load_and_preprocess_image(image_path, target_size=(256, 256)):
    
    print(f"Loading image: {image_path}")
    
    #Load .jp2 image from main directory dataset
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return None
    
    # Resize image to target size
    img_resized = cv2.resize(img, target_size)            #Resize to U-Net input size(256x256)
    img_rescaled = (img_resized / np.max(img_resized) * 255).astype(np.uint8)       #Normalize and convert to 8-bit
    return img_rescaled